In [100]:
import platform

if 'google.colab' in platform.sys.modules:
    !git clone https://github.com/humaidan/pysheets1.git

In [101]:
!pip install openpyxl


In [102]:
import openpyxl

if 'google.colab' in platform.sys.modules:
    from pysheets1.config import CONFIG
else:
    from config import CONFIG

In [103]:
# warnings.simplefilter("ignore", category=UserWarning)
debug = False

In [104]:
print("Opening source sheet ...")

try:
    if 'google.colab' in platform.sys.modules:
        wb = openpyxl.load_workbook("pysheets1/" + CONFIG["excelfile"])
    else:
        wb = openpyxl.load_workbook(CONFIG["excelfile"])

except Exception as e:
    print(f"Source workbook [{CONFIG['excelfile']}] error: {e}")
    raise  # Raise an exception to halt further code execution


Opening source sheet ...


In [105]:
# import os
# from google.colab import files


# excel_path = os.path.join(os.getcwd(), CONFIG["excelfile"])

# if not os.path.isfile(excel_path):
#     print("Excel file not found.")
#     print("Please upload the Excel file.")

#     # Prompt the user to upload the Excel file
#     uploaded_files = files.upload()

#     # Save the uploaded file
#     for filename in uploaded_files.keys():
#         os.rename(filename, excel_path)

#         wb = openpyxl.load_workbook(filename)
#         sheet1 = wb[CONFIG["sheetname_source"]]
#         sheet2 = wb[CONFIG["sheetname_dest"]]

#     print("Excel file uploaded and saved successfully.")



In [106]:
print("Parsing and writing data ...")

sheets = 0
start_row = CONFIG["start_row"]
sheet1 = wb[CONFIG["sheetname_source"]]
sheet2 = wb[CONFIG["sheetname_dest"]]

##########
# attributes = dir(sheet2.conditional_formatting._cf_rules)
# for attr in attributes:
#     if not attr.startswith('__'):
#         value = getattr(sheet2.conditional_formatting._cf_rules, attr)
#         print(f"{attr}: {value}")
#######
# print(sheet2.conditional_formatting._cf_rules.items())
# conditional_formatting1 = sheet2['C1'].conditional_formatting

# for row_no, row_val in enumerate(sheet1.iter_rows(min_row=start_row, max_row=12, values_only=True)):
for row_no, row_val in enumerate(sheet1.iter_rows(min_row=start_row, values_only=True)):
    if debug:
        print(f"- looping through record #{row_no+start_row}:")
    newsheet = wb.copy_worksheet(sheet2)
    newsheet.sheet_view.showGridLines = False

########################
    #copy conditional formatting
    # print(newsheet.conditional_formatting)
    # newsheet.conditional_formatting.add("C1", sheet2.conditional_formatting.rules[0]) #.update(sheet2.conditional_formatting.cf_rules)

    # conditional_formatting1 = sheet2.conditional_formatting
    # for rule in conditional_formatting1.rules:
    #     print(f'rule applied as {rule}')
    #     newsheet.conditional_formatting.add(newsheet['C1'].coordinate, rule)
    # newsheet.conditional_formatting._cf_rules.copy(sheet2.conditional_formatting._cf_rules)
    # newsheet.conditional_formatting._cf_rules

    
    # Copy conditional formatting rules from the source sheet to the destination sheet
    # for rule in sheet2.conditional_formatting._cf_rules:
    #     new_rule = deepcopy(rule)
    #     newsheet.conditional_formatting.add(rule.sqref, new_rule)

    # for rule in sheet2.conditional_formatting._cf_rules:
    #     sqref = deepcopy(rule.sqref)
    #     new_rule = deepcopy(rule)
    #     newsheet.conditional_formatting.add(sqref, new_rule)
        # for cell_range in sqref.split(","):
        #     cells = newsheet[cell_range]
        #     for cell in cells:
        #         newsheet.conditional_formatting.add(cell.coordinate, rule)

    newsheet.conditional_formatting._cf_rules = sheet2.conditional_formatting._cf_rules.copy()

##########################


    if debug:
        print(f"\t - set title to: {row_val[CONFIG['sysname_col']]} + {CONFIG['sysname_col_append']}:")

    newtitle = row_val[CONFIG["sysname_col"]].rstrip()
    if newtitle in CONFIG["short_titles"]:
        newtitle = CONFIG["short_titles"][newtitle]

    newtitle = newtitle.replace("- ", "")
    max_title_chars = CONFIG["max_title_chars"] - len(CONFIG["sysname_col_append"])
    if len(newtitle) > max_title_chars:
        newtitle = newtitle[: max_title_chars - 1] + "."

    newsheet.title = newtitle + CONFIG["sysname_col_append"]

    for target_cell, source_col_str in CONFIG["field_mapping"].items():
        # Read the value in "All Services" sheet in current row + config column
        source_value = ""
        if isinstance(source_col_str, str):
            source_col_val = openpyxl.utils.column_index_from_string(source_col_str)
            if debug:
                print(f"\t - sheet1.cell(row={row_no+start_row}, column={source_col_val}).value")
            source_value = sheet1.cell(row=row_no + start_row, column=source_col_val).value
            if source_value is None:
                source_value = ""

            if target_cell in CONFIG["pre_fills"] and source_value:
                source_value = CONFIG["pre_fills"][target_cell] + str(source_value)

            if debug:
                print(f"\t - reading AllSystems[row={row_no + start_row}, column={source_col_val}]  =  {source_value}.")

        else:  # list of str
            if debug:
                print(f"\t - detected a source list {source_col_str}:")

            source_value = ""
            for col in source_col_str:
                source_col_val = openpyxl.utils.column_index_from_string(col)
                cell = sheet1.cell(row=row_no + start_row, column=source_col_val).value
                if cell is not None:
                    source_value += "- " + str(cell) + "\n"

        # Write value in new sheet
        if debug:
            print(f"\t=Setting {newsheet.title} sheet cell [{target_cell}] to value: {source_value}.")
        if debug:
            print()
        newsheet[target_cell] = str(source_value).rstrip()

    sheets += 1


Parsing and writing data ...


In [107]:
print("Cleaning ...")

if 'google.colab' in platform.sys.modules:
    wb.save("pysheets1/" + CONFIG["modified_excelfile"])
else:
    wb.save(CONFIG["modified_excelfile"])



print(f"{sheets} new sheets added.")
print()


Cleaning ...
115 new sheets added.

